In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#import geopandas as gpd
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
train.head()

In [ ]:
train['keyword'].unique() 

In [ ]:
train['location'].unique()

*Finding tweets based on keyword 'accident'*

In [ ]:
train[train['keyword'] == 'accident'].head(7)

*Finding tweets based on location 'Canada'*

In [ ]:
train[train['location'] == 'Canada'].head(7)

*Distribution of train and test tweets*

In [ ]:
tweetLengthTrain = train['text'].str.len()
tweetLengthTest = test['text'].str.len()

plt.hist(tweetLengthTrain,bins=20,label='Train_Tweet')
plt.hist(tweetLengthTest,bins=20,label='Test_Tweet')
plt.legend()
plt.show()

*Target Distribution of keywords*

In [ ]:
train["target_mean"] = train.groupby("keyword")["target"].transform("mean")

plt.figure(figsize=(8,72))

sns.countplot(y=train.sort_values("target",ascending=False)["keyword"],hue=\
              train.sort_values("target",ascending=False)["target"])
plt.tick_params(axis="x",labelsize=15)
plt.tick_params(axis="y",labelsize=15)
plt.title("Target Distribution in Keywords")
plt.show()

In [ ]:
train_df=train
test_df=test

*Combining train and test datasets for furthur model evaluation*

In [ ]:
combine = train.append(test,ignore_index=True)
print('Shape of new Dataset:',combine.shape)
combine.tail()

In [ ]:
text_tweet = combine["text"]
text_tweet.tail()

In [ ]:
df =  pd.DataFrame(combine[['text']])
df.head(7)

*Removing punctuation from tweets*

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation]) #all punctuations 
    text = re.sub('[0-9]+', '', text) #all numbers
    return text

df['Tweet_punct'] = df['text'].apply(lambda x: remove_punct(x))
df.head(10)

In [ ]:
df["Tweet_punct"].drop_duplicates(inplace = True)
df["Tweet_punct"].shape

In [ ]:
combine["Tweet_punct"] = df['Tweet_punct']
combine.head()

In [ ]:
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import * 

In [ ]:
df = combine[combine['Tweet_punct'].notna()]
df.head()

*Tokenizing Tweets*

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
df.head(10)

In [ ]:
stemmer = PorterStemmer()
df['Tweet_tokenized'] = df['Tweet_tokenized'].apply(lambda x : [stemmer.stem(i) for i in x]  )
df.head()

*Joining the tokenized tweets*

In [ ]:
for i in range(len(df['Tweet_tokenized'])):
    df['Tweet_tokenized'][i] = ' '.join(df['Tweet_tokenized'][i])    
df['cleanedText'] = df['Tweet_tokenized']

In [ ]:
df.head()

In [ ]:
# Creating word Cloud for all Words in all tweets
from wordcloud import WordCloud
allWords = ' '.join([text for text in df['cleanedText']])
wordcloud = WordCloud(background_color='white', width=800, height=500, random_state=25, max_font_size=110).generate(allWords)
plt.figure(figsize=(10, 10)) 
plt.imshow(wordcloud, interpolation="bilinear") 
plt.axis('off') 
plt.show()

*Evaluating sentiment polarity of tweets using TextBlob*

In [ ]:
from textblob import TextBlob

def sentiment_polarity(text):
    tweet_text = TextBlob(str(text))
    sentiment_value = tweet_text.sentiment.polarity
    return sentiment_value

df['Tweet_Polarity'] = df['cleanedText'].apply(lambda x: sentiment_polarity(x.lower()))
df.head(10)

*Categorizing sentiments based on polarity value*

In [ ]:
def sentiment_analysis(value):
    sentiment=""
    if(value<0.0):
        sentiment = "negative"
    elif(value>0.0):
        sentiment = "positive"
    else:
        sentiment = "neutral"
    return sentiment

df['Tweet_Sentiments'] = df['Tweet_Polarity'].apply(lambda x: sentiment_analysis(float(x)))
df[["Tweet_punct","Tweet_Polarity","Tweet_Sentiments"]].head(10)

In [ ]:
import seaborn as sns
ax = sns.countplot(x="Tweet_Sentiments", data=df)

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['cleanedText'].apply(lambda x: tokenization(x.lower()))
df.head(10)

**Topic Modelling: LDA**

*Applying Dictionary of gensim to turn our tokenized documents into a id <-> term dictionary*

In [ ]:
from gensim.corpora import Dictionary

#create dictionary
text_dict = Dictionary(df.Tweet_tokenized)

#view integer mappings
text_dict.token2id

*convert tokenized documents into a document-term matrixa*

In [ ]:
tweets_bow = [text_dict.doc2bow(tweet) for tweet in df['Tweet_tokenized']]
tweets_bow

*Generate LDA model*

In [ ]:
from gensim.models.ldamodel import LdaModel

k = 10
tweets_lda = LdaModel(tweets_bow,
                      num_topics = k,
                      id2word = text_dict,
                      random_state = 1,
                      passes=10)

tweets_lda.show_topics()

In [ ]:
# word_count
train["word_count"] = train["text"].map(lambda x: len(str(x).split()))
test["word_count"] = test["text"].map(lambda x: len(str(x).split()))

# unique_word_count 
train["unique_word_count"] = train["text"].map(lambda x:len(set(str(x).split())))
test["unique_word_count"] = test["text"].map(lambda x:len(set(str(x).split())))

# stop_word_count
train["stop_word_count"] = train["text"].map(lambda x: len([elt for elt in str(x).lower().split() \
                                                                  if elt in stopwords.words("english")]))
test["stop_word_count"] = test["text"].map(lambda x:len([elt for elt in str(x).lower().split()\
                                                              if elt in stopwords.words("english")]))
# url_count 
train["url_count"] = train["text"].map(lambda x : len([w for w in str(x).lower().split()\
                                                            if 'http' or 'https' in w]))
test["url_count"] = test["text"].map(lambda x : len([w for w in str(x).lower().split()\
                                                          if "http" or "https" in w ]))
# mean_word_length
train["mean_word_length"] = train["text"].map(lambda x : np.mean([len(word) for word in x.split()]))
test["mean_word_length"] = test["text"].map(lambda x : np.mean([len(word) for word in x.split()]))

#char_count 
train["char_count"] = train["text"].map(lambda x:len(str(x)))
test["char_count"] = test["text"].map(lambda x:len(str(x)))

#punctuation_count
train["punctuation_count"] = train["text"].map(lambda x: len([elt for elt in str(x) if elt in string.punctuation]))
test["punctuation_count"] = test["text"].map(lambda x:len([elt for elt in str(x) if elt in string.punctuation]))
#hashtag_count
train["hashtag_count"] = train["text"].apply(lambda x:len([c for c in str(x) if c=="#"]))
test["hashtag_count"] = test["text"].apply(lambda x:len([c for c in str(x) if c=="#"]))

#mention_count
train["mention_count"] = train["text"].map(lambda x: len([c for c in str(x) if c=="@"]))
test["mention_count"] = test["text"].map(lambda x: len([c for c in str(x) if c=="@"]))

In [ ]:
train.head(4)

In [ ]:
# Heureunder, we will explore the distribution of each mega-feature per target and per dataset
# (train & test)
META_FEATURES = ["word_count","unique_word_count","stop_word_count","url_count",\
                 "mean_word_length","char_count","punctuation_count","hashtag_count","mention_count"]
fig,ax = plt.subplots(nrows=len(META_FEATURES),ncols=2,figsize=(20,50),dpi=100)
mask = train["target"]==1
for i,feature in enumerate(META_FEATURES):

    
   sns.distplot(train[mask][feature],ax=ax[i,0],label="Disaster",kde=False)
   sns.distplot(train[~mask][feature],ax=ax[i,0],label="Not Disaster",kde=False)
   ax[i,0].set_title("{} target distribution in trainning dataset".format(feature),fontsize=13)
 
   sns.distplot(train[feature],ax=ax[i,1],label="Train Dataset",kde=False)
   sns.distplot(test[feature],ax=ax[i,1],label="Test Dataset",kde=False)
   ax[i,1].set_title("{} training and test dataset distributions ".format(feature),fontsize=13)
   for j in range(2):
        ax[i,j].set_xlabel(" ")
        ax[i,j].tick_params(axis="x",labelsize=13)
        ax[i,j].tick_params(axis="y",labelsize=13)
        ax[i,j].legend()
plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(20,6))

train.groupby("target").count()["id"].plot(kind="pie",labels=["Not Disaster","Disaster"],\
                                              autopct="%1.1f pourcents",ax=ax[0])
sns.countplot(x=train["target"],hue=train["target"],ax=ax[1])
ax[1].set_xticklabels(["Non Disaster","Disaster"])
ax[0].tick_params(axis="x",labelsize=15)
ax[0].tick_params(axis="y",labelsize=15)
ax[0].set_ylabel("")
ax[1].tick_params(axis="x",labelsize=15)
ax[1].tick_params(axis="y",labelsize=15)
ax[0].set_title("Target distribution in training set",fontsize=13)
ax[1].set_title("Target count in training set",fontsize=13)

In [ ]:
# create the function which will be able to generate n_grams for each row of dataset.
def gen_n_grams(text,n_grams=1):
    """ This function allow to extract the n_gram in the introduced text.
    
      @param text(str): the text that the function, will use to extract features (n_grams).
      @param n_grams(int): the length of n_gram, that we will use.
      @return ngrams(list): list of the ngrams in the intriduced text.
    """
    tokens = [token for token in str(text).lower().split() if token not in stopwords.words("english")]
    ngrams = zip(*[tokens[i:] for i in range(n_grams)])
    
    return [" ".join(gram) for gram in ngrams]

# create the function which will be able to generate dataframe of n_gram features for disaster
# and non disaster tweets.
def gen_df_ngrams(n_grams=1):
    """ This function, allow to generate dataframes for n_grams in disaster tweets and non 
        disaster tweet
    """
    mask = train["target"]==1
    disaster_unigrams = defaultdict(int)
    non_disaster_unigrams = defaultdict(int)
    
    for tweet in train.loc[mask,"text"].values:
        for gram in gen_n_grams(tweet,n_grams=n_grams):
            disaster_unigrams[gram] +=1
    for tweet in train.loc[~mask,"text"].values:
        for gram in gen_n_grams(tweet,n_grams=n_grams):
            non_disaster_unigrams[gram] +=1
    df_disaster_n_grams = pd.DataFrame(sorted(disaster_unigrams.items(),reverse=True,key=\
                                              lambda item:item[1]))
    df_non_disaster_n_grams = pd.DataFrame(sorted(non_disaster_unigrams.items(),reverse=True,key=\
                                                  lambda item:item[1]))
    return df_disaster_n_grams,df_non_disaster_n_grams

# Define function, which allow to plot the N most occured n_gram in disaster tweet 
# and non disaster tweet.

def plot_ngrams(df_disaster_n_grams,df_non_disaster_unigrams,N=100,n_grams=1):
    """This function,allow to plot the top most n_grams in disaster tweet and non disaser tweet.
    """
    fig,ax = plt.subplots(1,2,figsize=(18,50))
    sns.barplot(y=df_disaster_n_grams[0].values[:N],x=df_disaster_n_grams[1].values[:N],ax=ax[0],\
               color="red")
    for i in range(2):
        ax[i].tick_params(axis="x",labelsize=15)
        ax[i].tick_params(axis="y",labelsize=15)
        ax[i].set_xlabel("Occurences")
        ax[i].spines["right"].set_visible(False)
    sns.barplot(y=df_non_disaster_unigrams[0].values[:N],x=df_non_disaster_unigrams[1].values[:N],\
               ax=ax[1],color="green")
    ax[0].set_title("Top most {} {}_grams for disaster tweets".format(N,n_grams),size=15)
    ax[1].set_title("Top most {} {}_grams for non disaster tweets".format(N,n_grams),size=15)

In [ ]:
df_disaster_unigrams,df_non_disaster_unigrams = gen_df_ngrams()
plot_ngrams(df_disaster_unigrams,df_non_disaster_unigrams)

In [ ]:
df_disaster_unigrams,df_non_disaster_unigrams = gen_df_ngrams(n_grams=2)
plot_ngrams(df_disaster_unigrams,df_non_disaster_unigrams,n_grams=2)

In [ ]:
# extract the most 100 bigrams per target (Disaster and not Disaster) when n_grams=3
df_disaster_unigrams,df_non_disaster_unigrams = gen_df_ngrams(n_grams=3)
plot_ngrams(df_disaster_unigrams,df_non_disaster_unigrams,n_grams=3)

In [ ]:
def process_text(text):
    """
    Removes punctuations(if any), stopwords and returns a list words
    """
    rm_pun = [char for char in text if char not in string.punctuation]
    rm_pun = ''.join(rm_pun)
    
    return [word for word in rm_pun.split() if word.lower() not in stopwords.words('english')]

In [ ]:
cv = CountVectorizer(analyzer=process_text).fit(train['text'])

In [ ]:
train_data=train
test_data=test

In [ ]:
print(len(cv.vocabulary_))

In [ ]:
text10 = train['text'][9]
text10

In [ ]:
cv10 = cv.transform([text10])
print(cv10)

In [ ]:
print(cv.get_feature_names()[5375])
print(cv.get_feature_names()[11579])
print(cv.get_feature_names()[11856])
print(cv.get_feature_names()[13190])
print(cv.get_feature_names()[25334])

In [ ]:
cv10.shape

In [ ]:
train_data_cv = cv.transform(train['text'])
print(train_data_cv.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer().fit(train_data_cv)

In [ ]:
tfidf10 = tfidf.transform(cv10)
print(tfidf10)

In [ ]:
train_data_tfidf = tfidf.transform(train_data_cv)
print(train_data_tfidf.shape)

*Naive Bayes Classifier Training the model*

In [ ]:
from sklearn.naive_bayes import MultinomialNB
target_model = MultinomialNB().fit(train_data_tfidf, train_data['target'])

In [ ]:
print('predicted:', target_model.predict(tfidf10)[0])
print('expected:', train_data.target[9])

*Predictions for test data test data vector and tf-idf*

In [ ]:
test_data_cv = cv.transform(test_data['text'])

test_data_tfidf = tfidf.transform(test_data_cv)

In [ ]:
target_model.predict(test_data_tfidf)